In [1]:
import rospy
from geometry_msgs.msg import Twist, Point, Quaternion
import tf
from math import radians, copysign, sqrt, pow, pi, atan2
from tf.transformations import euler_from_quaternion
import numpy as np
import time

In [2]:
rospy.init_node('ttb3_test', anonymous=False)
cmd_vel = rospy.Publisher('cmd_vel', Twist, queue_size=5)
position = Point()
move_cmd = Twist()
r = rospy.Rate(2)
tf_listener = tf.TransformListener()
odom_frame = 'odom'

In [3]:
try:
    tf_listener.waitForTransform(odom_frame, 'base_footprint', rospy.Time(), rospy.Duration(1.0))
    base_frame = 'base_footprint'
except (tf.Exception, tf.ConnectivityException, tf.LookupException):
    try:
        tf_listener.waitForTransform(odom_frame, 'base_link', rospy.Time(), rospy.Duration(1.0))
        base_frame = 'base_link'
    except (tf.Exception, tf.ConnectivityException, tf.LookupException):
        rospy.loginfo("Cannot find transform between odom and base_link or base_footprint")
        rospy.signal_shutdown("tf Exception")

In [4]:
def get_odom():
    global base_frame, odom_frame
    try: 
        (trans, rot) = tf_listener.lookupTransform(odom_frame, base_frame, rospy.Time(0))
        rotation = euler_from_quaternion(rot)

    except (tf.Exception, tf.ConnectivityException, tf.LookupException):
        rospy.loginfo("TF Exception")
        return

    return (Point(*trans), rotation[2])

In [5]:
def MoveRobot(distance, speed):

    global move_cmd, cmd_vel
    
    (position, rotation) = get_odom()
    init_x = position.x
    init_y = position.y

    movedDist = 0
    
    # As long as I am more than 5cm away, keep moving!
    while (distance - movedDist) > 0.05:   
        (position, _) = get_odom()   # Updat eodometry

        # Calculate euclidean distance
        movedDist = sqrt(pow(init_x - position.x, 2) + pow(init_y - position.y, 2))   
        
        move_cmd.linear.x = speed           # Set Speed
        move_cmd.angular.z = 0              # Make sure you don't rotate

        cmd_vel.publish(move_cmd) # Move
        
        r.sleep()  # use rospy sleep function to keep while loop running at desired rate


In [9]:
def RotateRobot(degrees, direction,speed=0.40):

    global move_cmd, cmd_vel, start_rot
    
    (_, init_angle) = get_odom()
    
    # Convert Initial angle to degrees!
    init_angle = math.degrees(init_angle)
    
    # If negative, it means the orientation of the robot is actually between 0 -> 180 degrees
    if init_angle < 0:
        init_angle*=-1

    # If positive, it means the orientation of the robot is actually between 180 - 360 degrees
    elif init_angle > 0:
        init_angle = 360 - init_angle
    
    # Keeps track of rotated degrees!
    rotatedDist = 0
    
    # As long as I am more than 5cm away, keep moving!
    while abs(degrees - rotatedDist) > 5:   
        
        (_, rotation) = get_odom()   # Update odometry
        
        # Convert current rotation to degrees!
        deg_rot = math.degrees(rotation)
        
        # If negative, it means the orientation of the robot is actually between 0 -> 180 degrees
        if deg_rot < 0:
            deg_rot*=-1
            
        # If positive, it means the orientation of the robot is actually between 180 - 360 degrees
        elif deg_rot > 0:
            deg_rot = 360 - deg_rot
        
        # Calculate hoe much I have rotated
        rotatedDist = deg_rot - init_angle
        
        # If rotatedDist is large and negative, it means we went through the conversion of 357,358,...,0,1 degrees
        if rotatedDist < -180:
            rotatedDist = 360 + rotatedDist
            
        # meanwhile if it is too large and positive, it means we went through the conversion of 3,2,...,358,357 degrees
        elif rotatedDist > 180:
            rotatedDist = 360 - rotatedDist
            
        rotatedDist = abs(rotatedDist)
            
        move_cmd.linear.x = 0                # Make sure you don't move
        move_cmd.angular.z = speed*direction  # Set Speed and direction

        cmd_vel.publish(move_cmd) # Move
        
        r.sleep()  # use rospy sleep function to keep while loop running at desired rate

In [1]:
# def RotateRobot(degrees, direction):

#     global move_cmd, cmd_vel, start_rot
    
#     (position, init_angle) = get_odom()

#     rotatedDist = 0
    
#     print("please rotate for radians = ", degrees)
#     print('initial angle = ', math.degrees(init_angle))
    
#     # As long as I am more than 5cm away, keep moving!
#     while abs(degrees - rotatedDist) > 0.1:   
#         (_, rotation) = get_odom()   # Updat eodometry
        
#         deg_rot = math.degrees(rotation)
        
#         # If negative, it means the orientation of the robot is actually between 0 -> 180 degrees
#         if deg_rot < 0:
#             deg_rot*=-1
            
#         # If positive, it means the orientation of the robot is actually between 180 - 360 degrees
#         elif deg_rot > 0:
#             deg_rot = 360 - deg_rot
        
#         print("current orientation ==> {} but actually is ==> {}".format(math.degrees(rotation), deg_rot))
        
#         # Calculate rotation angle
#         rotatedDist = math.deg(init_angle)
        
#         rotatedDist = abs(rotation - init_angle)  
        
#         move_cmd.linear.x = 0                # Make sure you don't move
#         move_cmd.angular.z = 0.15*direction  # Set Speed and direction

#         cmd_vel.publish(move_cmd) # Move
        
#         r.sleep()  # use rospy sleep function to keep while loop running at desired rate

In [7]:
def Move(distance=0.12,speed=0.4):
    
    global move_cmd, cmd_vel
    global moved_distance, stepCount
    
    direction = -1 if speed < 0 else 1
    
    # Obtain Odometry
    (position, rotation) = get_odom()
    init_x, init_y = position.x, position.y
    
    if stepCount <1: 
        speed = 0.2
        
    else:
        speed = 0.4
    
    #speed = speed * stepCount * stepCount
    
    # Move the robot in the direction and the distance defined above
    MoveRobot(distance, speed)
    
    # Add a sleep function to give time for the odomtry to update
    time.sleep(0.65)
    
    # Get new odometry
    (final_position, rotation) = get_odom()
    
    # Calculate travelled distance between initial position and final position
    moved_distance = sqrt(pow(init_x - final_position.x, 2) + pow(init_y - final_position.y, 2))*direction

    # Print the moved distance
    #print("moved distance = ", moved_distance)
    #print("\nX = {}, Y = {}, rotation = {}".format(final_position.x, final_position.y, rotation))

In [8]:
def Rotate(degrees=10, direction='left',speed=0.40):

    global move_cmd, cmd_vel
    global real_angle
    
    direction = 1 if direction=='left' else -1
    
    (_, init_angle) = get_odom()
    
    # Convert Initial angle to degrees!
    init_angle = math.degrees(init_angle)
    
    # If negative, it means the orientation of the robot is actually between 0 -> 180 degrees
    if init_angle < 0:
        init_angle*=-1

    # If positive, it means the orientation of the robot is actually between 180 - 360 degrees
    elif init_angle > 0:
        init_angle = 360 - init_angle
    
    # Move the robot in the direction and the distance defined above
    RotateRobot(degrees, direction, speed)
    
    # Add a sleep function to give time for the odomtry to update
    time.sleep(0.75)

    # Get new odometry
    (_, deg_rot) = get_odom()
    
    # Degrees
    deg_rot = math.degrees(deg_rot)
    
    # If negative, it means the orientation of the robot is actually between 0 -> 180 degrees
    if deg_rot < 0:
        deg_rot*=-1

    # If positive, it means the orientation of the robot is actually between 180 - 360 degrees
    elif deg_rot > 0:
        deg_rot = 360 - deg_rot
    
    # Calculate how much I have rotated
    rotatedDist = deg_rot - init_angle

    # If rotatedDist is large and negative, it means we went through the conversion of 357,358,...,0,1 degrees
    if rotatedDist < -180:
        rotatedDist = 360 + rotatedDist

    # meanwhile if it is too large and positive, it means we went through the conversion of 3,2,...,358,357 degrees
    elif rotatedDist > 180:
        rotatedDist = 360 - rotatedDist

    # Update real_angle
    rotated_angle = abs(rotatedDist)*direction
    
    # Update real angle
    real_angle-=rotated_angle
    
    # Make sure real_angle is between 0-360
    if real_angle/360>1:
        real_angle-=360
    elif real_angle<0:
        real_angle+=360

    # Print the rotated angle
    #print('rotated: {}. Real angle = {}'.format(rotated_angle, real_angle))

shutdown request: [/ttb3_test] Reason: new node registered with same name


In [9]:
def update_coordinates():
    
    global moved_distance, stepCount
    global real_angle, real_x, real_y
    
    if real_angle == 0: # Moved Forward ==> Y++
        real_y+=moved_distance
        
    elif real_angle == 90: # Moved Right ==> X++
        real_x+=moved_distance
        
    elif real_angle == 180: # Moved Backward ==> Y--
        real_y-=moved_distance
        
    elif real_angle == 270: # Moved Left ==> X--
        real_x-=moved_distance
        
    else:  # ==> Calculate X,Y components 
        delta_x = moved_distance*math.sin(math.radians(real_angle))
        delta_y = moved_distance*math.cos(math.radians(real_angle))
               
        real_x += delta_x
        real_y += delta_y
        
    logging.error("{},{},{},{}".format(real_x, real_y, real_angle, stepCount))